# TS Modeling

In [5]:
import pandas as pd

## Metrics

In [1]:
def MAPE(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def MPE(y_true, y_pred): 
    return np.mean((y_true - y_pred) / y_true) * 100



def weighted_MAPE(model, X_train, X_test, X_train_scaled, X_test_scaled, y_train, y_test):
    result = []
    for idx in X_train['id'].unique():
        
        idx_train = X_train[X_train['id'] == idx].index
        idx_test = X_test[X_test['id'] == idx].index
        
        X_train_id = X_train_scaled[idx_train]
        X_test_id = X_test_scaled[idx_test]

        y_train_id = y_train[idx_train]
        y_test_id = y_test[idx_test]

        train_prediction = model.predict(X_train_id)
        train_error = mean_absolute_percentage_error(train_prediction, y_train_id)
        
        test_prediction = model.predict(X_test_id)
        test_error_abs = mean_absolute_percentage_error(test_prediction, y_test_id)
        test_error_mean = mean_percentage_error(test_prediction, y_test_id)
        
        
        positive_err_rate  = sum((test_prediction - y_test_id) > 0)/ len(y_test_id)
        
        result.append([idx, train_error, test_error_abs, test_error_mean, y_test_id.sum(), positive_err_rate])
        
        
        
    result = pd.DataFrame(result, columns=['idx', 'train_error', 'test_error_abs', 
                                           'test_error_mean','sum_value', 'positive_err_rate'])
    result['weight'] = result['sum_value'] / result['sum_value'].sum()
    result['test_error_weighted'] = result['weight'] * result['test_error_mean']
    result['test_error_abs_weighted'] = result['weight'] * result['test_error_abs']
        
    return result





def MASE(training_series, testing_series, prediction_series):
    """
    Computes the MEAN-ABSOLUTE SCALED ERROR forcast error for univariate time series prediction.
    
    See "Another look at measures of forecast accuracy", Rob J Hyndman
    
    parameters:
        training_series: the series used to train the model, 1d numpy array
        testing_series: the test series to predict, 1d numpy array or float
        prediction_series: the prediction of testing_series, 1d numpy array (same size as testing_series) or float
        absolute: "squares" to use sum of squares and root the result, "absolute" to use absolute values.
    
    """
    print ("Needs to be tested.")
    n = training_series.shape[0]
    d = np.abs(np.diff( training_series) ).sum()/(n-1)
    
    errors = np.abs(testing_series - prediction_series )
    return errors.mean()/d


def SMAPE(y_true, y_pred): 
    return np.mean(np.abs(y_pred - y_true)/((np.abs(y_pred) + np.abs(y_true))/2))*100


In [2]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index].reset_index(drop=True)
    y_train = y.iloc[:test_index].reset_index(drop=True)
    X_test = X.iloc[test_index:].reset_index(drop=True)
    y_test = y.iloc[test_index:].reset_index(drop=True)
    
    return X_train, X_test, y_train, y_test


def plotModelResults(model, idx_train, idx_test, X_train, X_test, y_train, y_test, plot_intervals=False, plot_anomalies=False):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
    
    """
    
    warnings.filterwarnings('ignore')

    X_train = X_train[idx_train]
    X_test = X_test[idx_test]
    
    y_train = y_train[idx_train]
    y_test = y_test[idx_test]
    
    
    prediction = model.predict(X_test)
    
    plt.figure(figsize=(15, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_test.values, label="actual", linewidth=2.0)
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train, 
                                    cv=tscv, 
                                    scoring="neg_mean_absolute_error")
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)
        
        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)
        
        if plot_anomalies:
            anomalies = np.array([np.NaN]*len(y_test))
            anomalies[y_test<lower] = y_test[y_test<lower]
            anomalies[y_test>upper] = y_test[y_test>upper]
            plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    error = mean_absolute_percentage_error(prediction, y_test)
    plt.title("Mean absolute percentage error {0:.2f}%".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True);
    warnings.filterwarnings('default')
    

def plot_train_ModelResults(model, idx_train, idx_test, 
                            X_train, X_test, y_train, y_test, plot_intervals=False, plot_anomalies=False):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
    
    """
    
    warnings.filterwarnings('ignore')

    X_train = X_train[idx_train]
    X_test = X_test[idx_test]
    
    y_train = y_train[idx_train]
    y_test = y_test[idx_test]
    
    train_prediction = model.predict(X_train)
    prediction = model.predict(X_test)
    
    plt.figure(figsize=(15, 7))
    
    train_range = np.arange(len(y_train))
    
    plt.plot(train_range, train_prediction, "g", label="prediction_train",  alpha=0.5)
    plt.plot(train_range, y_train.values, label="actual_train", alpha=0.5)
    
    test_range = np.arange(len(y_test)) +  len(y_train)
    plt.plot(test_range, prediction, "g", label="prediction", linewidth=2.0, c='r')
    plt.plot(test_range, y_test.values, label="actual", linewidth=2.0)
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train, 
                                    cv=tscv, 
                                    scoring="neg_mean_absolute_error")
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)
        
        plt.plot(test_range, lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(test_range, upper, "r--", alpha=0.5)
        
        if plot_anomalies:
            anomalies = np.array([np.NaN]*len(y_test))
            anomalies[y_test<lower] = y_test[y_test<lower]
            anomalies[y_test>upper] = y_test[y_test>upper]
            plt.plot(test_range, anomalies, "o", markersize=10, label = "Anomalies")
    
    error = mean_absolute_percentage_error(prediction, y_test)
    plt.title("Mean absolute percentage error {0:.2f}%".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True);
    warnings.filterwarnings('default')
    
    
    
def plotCoefficients(model):
    """
        Plots sorted coefficient values of the model
    """
    
    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1).iloc[:25]
    
    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');
    


In [6]:
value_1_data = pd.read_pickle('../../data/3/data/value_1_data.pckl')

## Data preparation

In [7]:
# sort data
value_1_data = value_1_data.sort_values(['day', 'id'])

# train-test split
y = value_1_data.dropna()['value']
X = value_1_data.dropna().drop(['day', 'value'], axis=1)


X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

tscv = TimeSeriesSplit(n_splits=3)

NameError: name 'StandardScaler' is not defined

## Modeling

### Ridge

In [ ]:
%%time
ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)


plotCoefficients(ridge)

In [ ]:
prediction = ridge.predict(X_test_scaled)
error = mean_absolute_percentage_error(prediction, y_test)
print("Mean absolute percentage error {0:.2f}%".format(error))


prediction = ridge.predict(X_test_scaled)
error = mean_percentage_error(prediction, y_test)
print("Mean percentage error {0:.2f}%".format(error))


result_ridge = measure_qulity(ridge, X_train, X_test, X_train_scaled, X_test_scaled, y_train, y_test)
error = result_ridge['test_error_weighted'].sum()
print("Mean weighted absolute percentage error {0:.2f}%".format(error))
display(result_ridge.sort_values('weight', ascending=False).head(20))


In [ ]:


idx = 488
idx_train = X_train[X_train['id'] == idx].index
idx_test = X_test[X_test['id'] == idx].index
plot_train_ModelResults(ridge, idx_train, idx_test,
                 X_train_scaled, 
                 X_test_scaled,
                 y_train, 
                 y_test,
                 plot_intervals=True, plot_anomalies=True)

idx_train = X_train[X_train['id'] == idx].index
idx_test = X_test[X_test['id'] == idx].index
plotModelResults(ridge, idx_train, idx_test,
                 X_train_scaled, 
                 X_test_scaled,
                 y_train, 
                 y_test,
                 plot_intervals=True, plot_anomalies=True)


### Lasso

In [ ]:
%%time
lasso = LassoCV(cv=tscv)
lasso.fit(X_train_scaled, y_train)
plotCoefficients(lasso)

In [ ]:
prediction = lasso.predict(X_test_scaled)
error = mean_absolute_percentage_error(prediction, y_test)
print("Mean absolute percentage error {0:.2f}%".format(error))

prediction = lasso.predict(X_test_scaled)
error = mean_percentage_error(prediction, y_test)
print("Mean percentage error {0:.2f}%".format(error))


result_lasso = measure_qulity(lasso, X_train, X_test, X_train_scaled, X_test_scaled, y_train, y_test)
error = result_lasso['test_error_weighted'].sum()
print("Mean weighted absolute percentage error {0:.2f}%".format(error))
display(result_lasso.sort_values('weight', ascending=False).head(20))

In [ ]:
idx = 488
idx_train = X_train[X_train['id'] == idx].index
idx_test = X_test[X_test['id'] == idx].index
plot_train_ModelResults(lasso, idx_train, idx_test,
                 X_train_scaled, 
                 X_test_scaled,
                 y_train, 
                 y_test,
                 plot_intervals=True, plot_anomalies=True)


idx_train = X_train[X_train['id'] == idx].index
idx_test = X_test[X_test['id'] == idx].index
plotModelResults(lasso, idx_train, idx_test,
                 X_train_scaled, 
                 X_test_scaled,
                 y_train, 
                 y_test,
                 plot_intervals=True, plot_anomalies=True)

### Boosting

In [ ]:
%%time
lgbm_model = LGBMRegressor(n_estimators=1000, )
lgbm_model.fit(X_train_scaled, y_train)

In [ ]:
prediction = lgbm_model.predict(X_test_scaled)
error = mean_absolute_percentage_error(prediction, y_test)
print("Mean absolute percentage error {0:.2f}%".format(error))

prediction = lgbm_model.predict(X_test_scaled)
error = mean_percentage_error(prediction, y_test)
print("Mean percentage error {0:.2f}%".format(error))


result_lgbm = measure_qulity(lgbm_model, X_train, X_test, X_train_scaled, X_test_scaled, y_train, y_test)
error = result_lgbm['test_error_weighted'].sum()
print("Mean weighted absolute percentage error {0:.2f}%".format(error))
display(result_lgbm.sort_values('weight', ascending=False).head(20))

In [ ]:
idx = 488
idx_train = X_train[X_train['id'] == idx].index
idx_test = X_test[X_test['id'] == idx].index
plot_train_ModelResults(lgbm_model, idx_train, idx_test,
                 X_train_scaled, 
                 X_test_scaled,
                 y_train, 
                 y_test,
                 plot_intervals=True, plot_anomalies=True)

idx_train = X_train[X_train['id'] == idx].index
idx_test = X_test[X_test['id'] == idx].index
plotModelResults(lgbm_model, idx_train, idx_test,
                 X_train_scaled, 
                 X_test_scaled,
                 y_train, 
                 y_test,
                 plot_intervals=True, plot_anomalies=True)